In [22]:
import re
from wikimapper import WikiMapper

In [3]:
from templater3 import Templater
documents_to_learn = ['<b> spam and eggs </b>', '<b> ham and spam </b>',
                          '<b> white and black </b>'] # list of documents
template = Templater()
for document in documents_to_learn:
    template.learn(document)

print('Template created:', template._template)


document_to_parse = '<b> yellow and blue </b>'
print('Parsing other document:', template.parse(document_to_parse))

print('Filling the blanks:', template.join(['', 'red', 'orange', '']))

Template created: [None, '<b> ', None, ' and ', None, ' </b>', None]
Parsing other document: ['', 'yellow', 'blue', '']
Filling the blanks: <b> red and orange </b>


In [39]:
lines = open('simplewiki-20211120-lists-1k.tsv').readlines()
mapper = WikiMapper("index_enwiki-20190420.db")
for line in lines[:10]:
    file, word_list = line.split('\t')
    print(file)
    word_list = word_list.split('\\n')
    for word in word_list[0:5]:
        entities = re.findall(r'\[\[(.+?)\]\]',word)
        for entity in entities:
            try:
                entity = entity.replace(' ', '_')
                id = mapper.title_to_id(entity)
                print(entity, id)
            except:
                print(entity)
    print()

April
April 1 None
April Fools' Day None
April 1 None
Iran Q794
April 2 None
April 2 None
Thailand|Thai None
wikt:conservation|Conservation None
April 2 None
Autism Q38404

April
Easter Q21196
Christianity Q5043
March 15 None
April 18 None
March 19 None
April 22 None
March 20 None
April 23 None
Easter Q21196
March 22 None
April 25 None

April
April 1 None
1918 Q2094
Royal Air Force None
April 1 None
1976 Q2480
Apple Inc. None
April 1 None
1979 Q2484
Islamic Republic of Iran None
April 1 None
1999 Q2091
territory None
Nunavut Q2023
Canada Q16
April 1 None
2001 Q1988
Netherlands Q55
same-sex marriage None
country None

April
Christianity Q5043
Easter Q21196
March Q110
March Q110
May Q119
English language None
English language None
May Q119
June Q120
female None
Aries Q235939
March 21 None
April 20 None
Taurus Q111104
April 21 None
May 20 None
sweet pea None
Asteraceae|daisy None

August
August 1 None
Switzerland Q39
August 1 None
Benin Q962
August 1 None
Bermuda Q23635
Guyana Q734
Jamaic

In [41]:
mapper.id_to_titles("Q2403")

['March_15', '15_March', '15th_March', 'March_15th', 'Mar_15', '3/15', '15/3']